In [ ]:
%pip install -r requirements.txt

In [1]:
import numpy as np
from data_creator import DataCreator

data = np.array([2, 1, 3, 2, 1, 2, 3, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2])
data_modifier = DataCreator(data)
data_modifier.print_data_structure()
data_modifier.remove_data_classes(2)
data_modifier.print_data_structure()

det här är storleken 20
det här är hela grejen [2 1 3 2 1 2 3 2 3 1 2 3 1 2 3 1 2 3 1 2]
These are the classes [1 2 3]
det här är storleken 12
det här är hela grejen [1 3 1 3 3 1 3 1 3 1 3 1]
These are the classes [1 2 3]
